In [1]:
!pip install --upgrade transformers -q
!pip install accelerate -q

# GPTQ Dependencies
!pip install --upgrade optimum -q
!pip install --upgrade auto-gptq -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.7/417.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import sys, json

In [ ]:
model_id = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    padding=True,
    pad_side = "left",
    use_fast=True
)
tokenizer.pad_token = tokenizer.eos_token

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
    )

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [14]:
# Create a pipeline
pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True, # creative generation by discouraging greedy decoding
    temperature=0.75,
    top_p=0.95,
    return_full_text = False  # Only return the current output instead of returning complete prompt
    )

In [61]:
def parse_first_json(json_str):
    end_index = json_str.find("}") # Find the index of the first closing curly brace
    if end_index != -1:  # If a closing brace is found
        first_json_obj = json_str[:end_index + 1] # Parse the substring containing the first JSON object
        parsed_json = json.loads(first_json_obj) # Parse the JSON object
        return parsed_json
    else:
        return None

story = ""

for _ in range(10):

  system = f"""
  Starting from now, assume the role of an expert interactive fiction writer who specializes in crafting short, engaging, and captivating multiple-choice narratvies. \
  Your task is to generate a small narrative of no more than 50 words continuing the story so far provided to you in the square brackets. \
  Give 3 choices, that is, 3 possible continuations of the story for the current narrative. Every time you must only return a single JSON object with the follwoing three keys strictly following the provided description:
  {{
  "story": string, current generated narrative continuing the past story provided below in square brackets.
  "options": list, a list containing three string-type elements corresponding to the 3 options.
  }}
  Do not generate anything other than the JSON object inside {{}}.
  """

  assistant = f"""
  {{"story": "As you flee from the cursed room, you hear the mummy's voice growing louder. Sweat pouring down your face, you make a hasty decision.", "options": ["Seek help from the village elder.", "Search for a way to break the curse.", "Abandon your quest and leave the relics behind."]}}
  """

  user = f"""
  Start a new story if the story given below in the square brackets is empty, otherwise continue the story so far following the rules provided earlier.

  [{story}]

  """

  # one-shot prompting
  chat = [
    {"role": "user", "content": system},
    {"role": "assistant", "content": assistant},
    {"role": "user", "content": user}
  ]

  # prepare the prompt using the chat template
  prompt = tokenizer.apply_chat_template(chat, tokenize=False)
  # run the pipeline to generate the model output
  outputs = pipe(prompt)
  output = outputs[0]["generated_text"].strip()

  # sometimes the model generates notes/explanation after json so we only parse the first json object
  data = parse_first_json(output)
  narrative = data["story"]
  options = data["options"]
  print(narrative)
  [print(opt) for opt in options]
  user_input = input("> ")
  print("")
  story += data["story"] + user_input

You find yourself in a dense forest, surrounded by tall trees and the sounds of birds chirping. A sense of adventure fills you as you take your first steps into the unknown.
Follow a trail of footprints leading deeper into the forest.
Explore a nearby clearing filled with colorful wildflowers.
Set up camp and rest before continuing your journey.
> Explore a nearby clearing filled with colorful wildflowers.

As you reach the clearing, you notice a small pond in the center teeming with life. The sun casts a warm glow over the area, creating a serene atmosphere. A decision must be made.
Take a moment to rest and appreciate the beauty of nature.
Investigate the pond to see what creatures reside within.
Follow the sound of birds and explore further into the forest.
> Investigate the pond to see what creatures reside within.

The sun's gentle rays illuminate the water's surface, revealing a variety of aquatic creatures. You lean closer to get a better look.
Attempt to catch a fish for dinner

In [62]:
print(prompt)

<s>[INST] Starting from now, assume the role of an expert interactive fiction writer who specializes in crafting short, engaging, and captivating multiple-choice narratvies..
  Your task is to generate a small narrative of no more than 50 words continuing the story so far provided to you in the square brackets.
  Give 3 choices, that is, 3 possible continuations of the story for the current narrative. Every time you must only return a single JSON object with the follwoing three keys strictly following the provided description:
  {
  "story": string, current generated narrative continuing the past story provided below in square brackets.
  "options": list, a list containing three string-type elements corresponding to the 3 options.
  }
  Do not generate anything other than the JSON object inside {}.
   [/INST]
  {"story": "As you flee from the cursed room, you hear the mummy's voice growing louder. Sweat pouring down your face, you make a hasty decision.", "options": ["Seek help from th

In [64]:
story

"You find yourself in a dense forest, surrounded by tall trees and the sounds of birds chirping. A sense of adventure fills you as you take your first steps into the unknown.Explore a nearby clearing filled with colorful wildflowers.As you reach the clearing, you notice a small pond in the center teeming with life. The sun casts a warm glow over the area, creating a serene atmosphere. A decision must be made.Investigate the pond to see what creatures reside within.The sun's gentle rays illuminate the water's surface, revealing a variety of aquatic creatures. You lean closer to get a better look.Attempt to catch a fish for dinner.As you lean closer to the pond, a large fish jumps out, startling you. You quickly grab for your fishing rod, but remember you left it behind in your haste to explore.Try to catch a fish with your bare hands.As you ponder your options, a sudden rustling in the bushes nearby makes you turn around. The forest seems to come alive with uncertainty, your heart pound

In [67]:
output

'{ "story": "As the rustling in the bushes grows louder, your heart races with fear. You reach for a nearby stick to use as a makeshift weapon. The forest seems to come alive with uncertainty, your heart pounding in your chest. Approach the bushes cautiously and investigate.", "options": ["Discover a friendly forest creature.", "Encounter a dangerous predator.", "Realize it was just the wind and continue exploring the clearing."] }'